In [ ]:
import numpy as np
import pandas as pd

Załadowanie plików źródłowych:


*   Property_sales_data_New_York.csv
*   Airbnb_data_New_York.csv



In [ ]:
sales_NY = pd.read_csv("dane_zrodlowe/Property_sales_data_New_York.csv")
airbnb_NY = pd.read_csv("dane_zrodlowe/Airbnb_data_New_York.csv")

Uporządkowanie ramki danych ze sprzedażą mieszkań w Nowym Jorku - sales_NY



In [ ]:
# Usunięcie zbędnych kolumn
try:
    sales_NY = sales_NY.drop(columns=["Unnamed: 0"])
except KeyError():
    pass

# Usunięcie zduplikwowanych wierszy
sales_NY.drop_duplicates()

# Dodanie nowej kolumny z dzielnicą
brough_names = {
    1: "Manhattan",
    2: "Bronx",
    3: "Brooklyn",
    4: "Queens",
    5: "Staten Island",
}

for i in list(brough_names.keys()):
    sales_NY.loc[sales_NY["BOROUGH"] == i, "BOROUGH NAME"] = brough_names[i]

# Modyfikacja kolumn tekstowych na numeryczne
sales_NY["SALE PRICE"] = sales_NY["SALE PRICE"].str.replace("\D", "", regex=True)
sales_NY = sales_NY[sales_NY["SALE PRICE"] != ""]
sales_NY = sales_NY.reset_index()

sales_NY["GROSS SQUARE FEET"] = sales_NY["GROSS SQUARE FEET"].str.replace(
    "\D", "", regex=True
)
sales_NY = sales_NY[
    np.logical_and(
        sales_NY["GROSS SQUARE FEET"] != "", sales_NY["GROSS SQUARE FEET"] != "0"
    )
]
sales_NY = sales_NY.reset_index()
sales_NY = sales_NY.drop(columns=["level_0"])

sales_NY["LAND SQUARE FEET"] = sales_NY["LAND SQUARE FEET"].str.replace(
    "\D", "", regex=True
)
sales_NY = sales_NY[
    np.logical_and(
        sales_NY["LAND SQUARE FEET"] != "", sales_NY["LAND SQUARE FEET"] != "0"
    )
]

sales_NY = sales_NY.reset_index()
try:
    sales_NY = sales_NY.drop(columns=["level_0"])
except KeyError():
    pass

for column in list(sales_NY.columns):
    try:
        sales_NY[column] = sales_NY[column].apply(pd.to_numeric)
    except ValueError():
        pass

# Odfiltrowanie niewłaściwych wartości ceny równej 0
sales_NY = sales_NY[sales_NY["SALE PRICE"] > 0]
sales_NY = sales_NY.reset_index()

# Dodanie kolumny z ceną stopy kwadratowej ("PRICE PER SQUARE FT")
sales_NY["PRICE PER SQUARE FT"] = round(
    sales_NY["SALE PRICE"] / sales_NY["GROSS SQUARE FEET"], 2
)

# Dodanie kolumny z dzielnicami do dalszego porównania
sales_NY["Neighborhood"] = sales_NY["NEIGHBORHOOD"].astype(str)
sales_NY["Neighborhood"] = sales_NY["Neighborhood"].str.replace("'", "", regex=True)
sales_NY["Neighborhood"] = sales_NY["Neighborhood"].str.replace("-", " ", regex=True)

# Usunięcie zbędnych kolumn
try:
    sales_NY = sales_NY.drop(columns=["level_0"])
except KeyError():
    try:
        sales_NY = sales_NY.drop(columns=["index"])
    except KeyError():
        pass

sales_NY = sales_NY.reset_index()

sales_NY.head(10)

Uporządkowanie ramki danych z wynajmem krótkoterminowym w Nowym Jorku - airbnb_NY

In [ ]:
# Usunięcie zduplikwowanych wierszy
airbnb_NY.drop_duplicates()

# Odfiltrowanie zerowych wartości
airbnb_NY = airbnb_NY[
    np.logical_and(airbnb_NY["availability_365"] > 0, airbnb_NY["price"] > 0)
]
airbnb_NY = airbnb_NY.reset_index()

# Uporządkowanie kolumny z dzielnicą
airbnb_NY["Neighborhood"] = airbnb_NY["neighbourhood"].astype(str)
airbnb_NY["Neighborhood"] = airbnb_NY["Neighborhood"].str.replace("'", "", regex=True)
airbnb_NY["Neighborhood"] = airbnb_NY["Neighborhood"].str.replace("-", " ", regex=True)
airbnb_NY["Neighborhood"] = airbnb_NY["Neighborhood"].str.upper()

# Usunięcie zbędnych kolumn
try:
    airbnb_NY = airbnb_NY.drop(columns=["level_0"])
except KeyError():
    try:
        airbnb_NY = airbnb_NY.drop(columns=["index"])
    except KeyError():
        pass

airbnb_NY = airbnb_NY.reset_index()

airbnb_NY.head(10)

Ujednolicenie danych w kolumnach z dzielnicami w obu ramkach danych, żeby było możliwe ich porównanie

In [ ]:
Neighborhood_dict = {
    "BAY TERRACE": "BAY TERRACE, STATEN ISLAND",
    "CASTLE HILL": "CASTLE HILL/UNIONPORT",
    "UNIONPORT": "CASTLE HILL/UNIONPORT",
    "FLATBUSH CENTRAL": "FLATBUSH",
    "FLATBUSH EAST": "EAST FLATBUSH",
    "FLATBUSH LEFFERTS GARDEN": "FLATBUSH",
    "FLATBUSH NORTH": "FLATBUSH",
    "FLUSHING NORTH": "FLUSHING",
    "FLUSHING SOUTH": "FLUSHING",
    "FLATIRON DISTRICT": "FLATIRON",
    "FINANCIAL DISTRICT": "FINANCIAL",
    "GREENWICH VILLAGE CENTRAL": "GREENWICH VILLAGE",
    "GREENWICH VILLAGE WEST": "GREENWICH VILLAGE",
    "EAST HARLEM": "HARLEM EAST",
    "HARLEM CENTRAL": "HARLEM",
    "HARLEM UPPER": "HARLEM",
    "HARLEM WEST": "HARLEM",
    "HIGHBRIDGE": "HIGHBRIDGE/MORRIS HEIGHTS",
    "MORRIS HEIGHTS": "HIGHBRIDGE/MORRIS HEIGHTS",
    "KINGSBRIDGE": "KINGSBRIDGE/JEROME PARK",
    "MELROSE": "MELROSE/CONCOURSE",
    "CONCOURSE": "MELROSE/CONCOURSE",
    "MIDTOWN CBD": "MIDTOWN",
    "MIDTOWN EAST": "MIDTOWN",
    "MIDTOWN WEST": "MIDTOWN",
    "VAN NEST": "MORRIS PARK/VAN NEST",
    "MORRIS PARK": "MORRIS PARK/VAN NEST",
    "MOUNT HOPE": "MOUNT HOPE/MOUNT EDEN",
    "MOUNT EDEN": "MOUNT HOPE/MOUNT EDEN",
    "MORRISANIA": "MORRISANIA/LONGWOOD",
    "LONGWOOD": "MORRISANIA/LONGWOOD",
    "MOTT HAVEN": "MOTT HAVEN/PORT MORRIS",
    "PORT MORRIS": "MOTT HAVEN/PORT MORRIS",
    "SCHUYLERVILLE": "SCHUYLERVILLE/PELHAM BAY",
    "PELHAM BAY": "SCHUYLERVILLE/PELHAM BAY",
    "UPPER EAST SIDE (59 79)": "UPPER EAST SIDE",
    "UPPER EAST SIDE (79 96)": "UPPER EAST SIDE",
    "UPPER EAST SIDE (96 110)": "UPPER EAST SIDE",
    "UPPER WEST SIDE (59 79)": "UPPER WEST SIDE",
    "UPPER WEST SIDE (79 96)": "UPPER WEST SIDE",
    "UPPER WEST SIDE (96 116)": "UPPER WEST SIDE",
    "WASHINGTON HEIGHTS LOWER": "WASHINGTON HEIGHTS",
    "WASHINGTON HEIGHTS UPPER": "WASHINGTON HEIGHTS",
    "WILLIAMSBURG CENTRAL": "WILLIAMSBURG",
    "WILLIAMSBURG EAST": "WILLIAMSBURG",
    "WILLIAMSBURG NORTH": "WILLIAMSBURG",
    "WILLIAMSBURG SOUTH": "WILLIAMSBURG",
}

for i in list(Neighborhood_dict.keys()):
    if i in set(sales_NY["Neighborhood"]):
        sales_NY.loc[sales_NY["Neighborhood"] == i, "Neighborhood"] = Neighborhood_dict[
            i
        ]
    elif i in set(airbnb_NY["Neighborhood"]):
        airbnb_NY.loc[airbnb_NY["Neighborhood"] == i, "Neighborhood"] = (
            Neighborhood_dict[i]
        )

Neighborhoods_list = list(
    set(sales_NY["Neighborhood"]).intersection(set(airbnb_NY["Neighborhood"]))
)
Neighborhoods_list.sort()

# Select only wanted columns in sales_NY and fix format of columns names and data in columns
sales_NY = sales_NY[
    [
        "BOROUGH NAME",
        "Neighborhood",
        "BUILDING CLASS CATEGORY",
        "TAX CLASS AT PRESENT",
        "BLOCK",
        "LOT",
        "EASE-MENT",
        "BUILDING CLASS AT PRESENT",
        "ADDRESS",
        "APARTMENT NUMBER",
        "ZIP CODE",
        "RESIDENTIAL UNITS",
        "COMMERCIAL UNITS",
        "TOTAL UNITS",
        "LAND SQUARE FEET",
        "GROSS SQUARE FEET",
        "YEAR BUILT",
        "TAX CLASS AT TIME OF SALE",
        "BUILDING CLASS AT TIME OF SALE",
        "SALE PRICE",
        "SALE DATE",
        "PRICE PER SQUARE FT",
    ]
].copy()

sales_NY = sales_NY.rename(str.title, axis="columns")
sales_NY = sales_NY.rename(columns={"Borough Name": "Borough"})
sales_NY["Address"] = sales_NY["Address"].str.title()
sales_NY["Building Class Category"] = sales_NY["Building Class Category"].str.title()
sales_NY["Sale Date"] = pd.to_datetime(sales_NY["Sale Date"])

for i in range(0, len(sales_NY["Neighborhood"]) - 1):
    if sales_NY["Neighborhood"][i] not in Neighborhoods_list:
        sales_NY = sales_NY.drop(index=i)
sales_NY = sales_NY.reset_index()
sales_NY["Neighborhood"] = sales_NY["Neighborhood"].str.title()
sales_NY = sales_NY.drop(columns=["index"])

# Select only wanted columns in airbnb_NY and fix format of columns names and data in columns
airbnb_NY = airbnb_NY[
    [
        "Neighborhood",
        "name",
        "host_id",
        "host_name",
        "neighbourhood_group",
        "latitude",
        "longitude",
        "room_type",
        "price",
        "minimum_nights",
        "number_of_reviews",
        "last_review",
        "reviews_per_month",
        "calculated_host_listings_count",
        "availability_365",
    ]
].copy()

airbnb_NY = airbnb_NY.rename(columns={"neighbourhood_group": "Borough"})
airbnb_NY.columns = airbnb_NY.columns.str.replace("_", " ", regex=True).str.title()
for i in range(0, len(airbnb_NY["Neighborhood"]) - 1):
    if airbnb_NY["Neighborhood"][i] not in Neighborhoods_list:
        airbnb_NY = airbnb_NY.drop(index=i)
airbnb_NY = airbnb_NY.reset_index()
airbnb_NY["Neighborhood"] = airbnb_NY["Neighborhood"].str.title()
airbnb_NY = airbnb_NY.drop(
    index=airbnb_NY.loc[airbnb_NY["Neighborhood"] == "Hells Kitchen"].index
)

airbnb_NY = airbnb_NY.drop(columns=["index"])

Statystyki tabeli sales_NY

In [ ]:
# Sprawdzenie, czy wszystkie niechciane puste komórki zostały usunięte z sales_NY
sales_NY.isnull().sum()

In [ ]:
# Podsumowanie każdej kolumny z sales_NY
sales_NY.describe()

In [ ]:
sales_NY.head(10)

Statystyki tabeli airbnb_NY

In [ ]:
# Sprawdzenie, czy wszystkie niechciane puste komórki zostały usunięte z airbnb_NY
airbnb_NY.isnull().sum()

In [ ]:
# Podsumowanie każdej kolumny z airbnb_NY
airbnb_NY.describe()

In [ ]:
airbnb_NY.head(10)

In [ ]:
# Zapis wyczyszczonych tabeli jako .json do użycia w aplikacji 2_AplikacjaStreamlit.py
sales_NY.to_json("dane_przeksztalcone/sales_NY.json")
airbnb_NY.to_json("dane_przeksztalcone/airbnb_NY.json")